In [1]:
!pip install gym==0.10.5
!git clone https://github.com/openai/multiagent-particle-envs.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 35.0 MB/s 
     |████████████████████████████████| 1.1 MB 53.3 MB/s 
  Created wheel for gym: filename=gym-0.10.5-py3-none-any.whl size=1581307 sha256=e6bff3bcb00fb2b0beacbcdbdb66aafd74be39c2af579850501724ed68a97267
  Stored in directory: /root/.cache/pip/wheels/7a/2c/df/a05b548a40fae16ca400ecbeda0067e1a296499c1fbd7e0c9a
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
Cloning into 'multiagent-particle-envs'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 242 (delta 0), reused 3 (delta 0), pack-reused 237
Receiving objects: 100% (242/242), 107.24 KiB | 477.00 KiB/s, done.
Resolving deltas: 100% (127/127), done.


In [2]:
cd multiagent-particle-envs

/content/multiagent-particle-envs


In [3]:
pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/multiagent-particle-envs
  Running setup.py develop for multiagent


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
from torch.autograd import variable as v
from make_env import make_env
env = make_env("simple_reference")

In [ ]:
class Actor(nn.Module):
  def __init__(self,state_size,action_size):
    super(Actor,self).__init__()
    self.state_size  = state_size
    self.action_size = action_size
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.fc3 = nn.Linear(64,128).to(self.device)
    self.fc4 = nn.Linear(128,256).to(self.device)
    self.fc5 = nn.Linear(256,512).to(self.device)
    self.fc6 = nn.Linear(512,256).to(self.device)
    self.fc7 = nn.Linear(256,128).to(self.device)
    self.fc8 = nn.Linear(128,64).to(self.device)
    self.fc9 = nn.Linear(64,32).to(self.device)
    self.fc10= nn.Linear(32,self.action_size).to(self.device)
    
  def forward(self,state,message):
    message   = torch.tensor([message.item(),0]).to(self.device)
    self.fs1 = nn.Linear(state.shape[0],32).to(self.device)
    self.fm1 = nn.Linear(message.shape[0],32).to(self.device)
    state    = torch.tensor(state).to(self.device)
    s1       = self.fs1(state).to(self.device)
    m1       = self.fm1(message)
    cat      = torch.cat([s1.view(32,-1),m1.view(32,-1)],0)
    cat      = torch.reshape(cat,(-1,)).to(self.device)
    self.fc2 = nn.Linear(cat.shape[0],64).to(self.device)
    x        = self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(self.fc2(cat)))))))
    x        = self.fc9(f.relu(self.fc8(f.relu(self.fc7(f.relu(self.fc6(f.relu(x))))))))
    x        = f.softmax(self.fc10(f.relu(x)))
    return x

In [ ]:

class Critic(nn.Module):
  def __init__(self,state_size,action_size,message_size):
    super(Critic,self).__init__()
    self.state_size  = state_size
    self.action_size = action_size
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.fc3 = nn.Linear(64,128).to(self.device)
    self.fc4 = nn.Linear(128,256).to(self.device)
    self.fc5 = nn.Linear(256,512).to(self.device)
    self.fc6 = nn.Linear(512,256).to(self.device)
    self.fc7 = nn.Linear(256,128).to(self.device)
    self.fc8 = nn.Linear(128,64).to(self.device)
    self.fc9 = nn.Linear(64,32).to(self.device)
    self.fc10= nn.Linear(32,1).to(self.device)
  def forward(self,state,action,reward):
    self.fs1 = nn.Linear(state.shape[0],32).to(self.device)
    self.fa1 = nn.Linear(action.shape[0],32).to(self.device)
    self.fr1 = nn.Linear(2,32).to(self.device)
    s1       = self.fs1(state)
    a1       = self.fa1(action)
    r1       = self.fr1(reward)
    cat      = torch.cat([s1.view(32,-1),r1.view(32,-1),a1.view(32,-1)],0)
    cat      = torch.reshape(cat,(-1,))
    self.fc2 = nn.Linear(cat.shape[0],64).to(self.device)
    x        = self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(self.fc2(cat)))))))
    x        = self.fc9(f.relu(self.fc8(f.relu(self.fc7(f.relu(self.fc6(f.relu(x))))))))
    x        = self.fc10(f.relu(x))
    return x

In [ ]:
class  MGN(nn.Module):
  def __init__(self,state_size,action_size,message_size):
    super(MGN,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.message_size = message_size
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.fo3 = nn.Linear(64,128).to(self.device)
    self.fo4 = nn.Linear(128,256).to(self.device)
    self.fo5 = nn.Linear(256,512).to(self.device)
    self.fo6 = nn.Linear(512,256).to(self.device)
    self.fo7 = nn.Linear(256,128).to(self.device)
    self.fo8 = nn.Linear(128,64).to(self.device)
    self.fo9 = nn.Linear(64,32).to(self.device)
    self.f10 = nn.Linear(32,21).to(self.device)
  def action_predictor(self,observation):
    self.fa1 = nn.Linear(observation.shape[0],32).to(self.device)
    self.fa2 = nn.Linear(32,64).to(self.device)
    self.fa3 = nn.Linear(64,128).to(self.device)
    self.fa4 = nn.Linear(128,256).to(self.device)
    self.fa5 = nn.Linear(256,512).to(self.device)
    self.fa6 = nn.Linear(512,256).to(self.device)
    self.fa7 = nn.Linear(256,128).to(self.device)
    self.fa8 = nn.Linear(128,64).to(self.device)
    self.fa9 = nn.Linear(64,32).to(self.device)
    self.fa10= nn.Linear(32,self.action_size).to(self.device)
    obs = self.fa1(observation)
    x   = self.fa5(f.relu(self.fa4(f.relu(self.fa3(f.relu(self.fa2(f.relu(obs))))))))
    x   = self.fa9(f.relu(self.fa8(f.relu(self.fa7(f.relu(self.fa6(f.relu(x))))))))
    x   = f.softmax(self.fa10(f.relu(x)))
    return x
  def observation_predictor(self,prsent_action,other_action,observation):
    self.fo1 = nn.Linear(observation.shape[0],32).to(self.device)
    self.fa1 = nn.Linear(prsent_action.shape[0],32).to(self.device)
    self.fa2 = nn.Linear(other_action.shape[0],32).to(self.device)
    o        = self.fo1(observation)
    at       = self.fa1(prsent_action)
    at1      = self.fa2(other_action)
    cat      = torch.cat([o.view(32,-1),at.view(32,-1),at1.view(32,-1)],0)
    cat      = torch.reshape(cat,(-1,))
    self.fo2 = nn.Linear(cat.shape[0],64).to(self.device)
    x        = self.fo5(f.relu(self.fo4(f.relu(self.fo3(f.relu(self.fo2(cat)))))))
    x        = self.fo9(f.relu(self.fo8(f.relu(self.fo7(f.relu(self.fo6(x)))))))
    x        = f.relu(self.f10(f.relu(x)))
    return x
  def policy(self,message,observation):
    self.fm1  = nn.Linear(message.shape[0],32).to(self.device)
    self.fs1  = nn.Linear(observation.shape[0],32).to(self.device)
    m         = self.fm1(message)
    o         = self.fs1(observation)
    cat       = torch.cat([m.view(32,-1),o.view(32,-1)],0)
    cat       = torch.reshape(cat,(-1,))
    self.fc1  = nn.Linear(cat.shape[0],64).to(self.device)
    self.fc2  = nn.Linear(64,128).to(self.device)
    self.fc3  = nn.Linear(128,256).to(self.device)
    self.fc4  = nn.Linear(256,512).to(self.device)
    self.fc5  = nn.Linear(512,256).to(self.device)
    self.fc6  = nn.Linear(256,128).to(self.device)
    self.fc7  = nn.Linear(128,64).to(self.device)
    self.fc8  = nn.Linear(64,32).to(self.device)
    self.fc9  = nn.Linear(32,self.action_size).to(self.device)
    x         = self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(self.fc2(f.relu(self.fc1(cat)))))))))
    x         = self.fc8(f.relu(self.fc7(f.relu(self.fc6(f.relu(x))))))
    x         = f.softmax(self.fc9(f.relu(x)))
    return x
  def IGTM(self,message,observation,action):
    other_action   = self.action_predictor(observation)
    observation_t1 = self.observation_predictor(action,other_action,observation)
    policy         = self.policy(message,observation_t1)
    return message,observation_t1,policy
  def attention(self,message,observation,policy):
    trajectory    = torch.cat([observation,policy],0)
    trajectory    = torch.reshape(trajectory , (-1,))
    self.wv       = nn.Linear(trajectory.shape[0],128).to(self.device)
    self.wk       = nn.Linear(trajectory.shape[0],128).to(self.device)
    self.wq       = nn.Linear(message.shape[0],128).to(self.device)
    v             = self.wv(trajectory)
    k             = self.wk(trajectory)
    q             = self.wq(message)
    x1            = torch.matmul(q,k)
    x2            = f.softmax(x1)
    x1            = torch.tensor([x1.item(),0])
    x2            = torch.tensor([x2.item(),0])
    x             = torch.matmul(x1,x2)
    return x #message
  def forward(self,observation,action,message):
    message                     = torch.tensor([message.item(),0]).to(self.device)
    msg,observationt2,policy    =  self.IGTM(message,observation,action)
    msg                         =  self.attention(msg,observationt2,policy)
    return msg

In [ ]:
class Buffer:
  def __init__(self):
    self.agent1_reward    = []
    self.agent2_reward    = []
    self.agent1_returns   = []
    self.agent2_returns   = []
    self.agent1_q_value   = []
    self.agent2_q_value   = []
    self.agent1_n_q_value = []
    self.agent2_n_q_value = []
    self.agent1_loss      = []
    self.agent2_loss      = []
    self.agent1_mgn_loss  = []
    self.agent2_mgn_loss  = []
    self.agent1_msg       = []
    self.agent2_msg       = []
    self.agent1_action    = []
    self.agent2_action    = []
    self.agent1_prob      = []
    self.agent2_prob      = []
    self.agent1_next_prob = []
    self.agent2_next_prob = []
    self.agent1_actor_loss= []
    self.agent2_actor_loss= []
    self.agent1_critic_loss=[]
    self.agent2_critic_loss=[]
    self.agent1_mean_reward=[]
    self.agent2_mean_reward=[]
    self.agent1_mean_critic=[]
    self.agent2_mean_critic=[]
    self.agent1_mean_loss = []
    self.agent2_mean_loss = []
    self.agent1_mean_mgn  = []
    self.agent2_mean_mgn  = []
    self.agent1_mean_actor= []
    self.agent2_mean_actor= [] 
    self.agent1_mean_value= []
    self.agent2_mean_value= []
    self.agent1_meannvalue= []
    self.agent2_meannvalue= []
    self.agent1_meanreturn= []
    self.agent2_meanreturn= []
    self.agent1_meanlog   = []
    self.agent2_meanlog   = []
    self.agent1_nextlog   = []
    self.agent2_nextlog   = []
    self.episodes         = []
    self.loss             = []
    self.mean_loss        = []
    self.ag1r             = []
    self.ag2r             = []

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
from torch.autograd import variable as v
import sys
sys.path.append('./')
from Networks.Actor import Actor
from Networks.Critic import Critic
from Networks.mgn import MGN

class Agent1:
  def __init__(self,state_size,action_size,message_size,buffer):
    self.state_size = state_size
    self.action_size  = action_size
    self.message_size = message_size
    self.gamma        = 0.99
    self.lamda        = 0.95
    self.lr1          = 0.0000009
    self.lr2          = 0.0000007
    self.buffer       = buffer
    self.device       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor        = Actor(self.state_size,self.action_size).to(self.device)
    self.critic       = Critic(self.state_size,self.action_size,self.message_size).to(self.device)
    self.mgn          = MGN(self.state_size,self.action_size,self.message_size).to(self.device)
    self.actor_optim  = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim = optim.Adam(self.critic.parameters() ,lr = self.lr2)
    self.mgn_optim    = optim.Adam(self.mgn.parameters(),lr = self.lr2)
  def choose_action(self,observation,message):
    act               = self.actor(observation,message).to(self.device)
    return act
  def q_value(self,observation,action,reward):
    reward            = torch.tensor([reward.item(),0],dtype = torch.float32).to(self.device)
    value             = self.critic(observation,action,reward).to(self.device)
    return value
  def message(self,observation,action,message):
    message           = self.mgn(observation,action,message).to(self.device)
    return message
  def ppo_iter(self,reward,value,next_value,done):
    returns   = []
    gae       = 0
    for i in range(5):
      delta   = reward + self.gamma *(1-done)* next_value - value
      gae     = delta + self.gamma*self.lamda*gae*(1-done)
      returns.insert(0,gae + value+next_value)
    return returns
  def appending(self,loss,actor_loss,critic_loss,mgn,value,next_value,log_prob,returns,next_log_prob,action,msg1):
    self.buffer.agent1_mgn_loss.append(mgn)
    self.buffer.agent1_actor_loss.append(actor_loss)
    self.buffer.agent1_critic_loss.append(critic_loss)
    self.buffer.agent1_loss.append(loss)
    self.buffer.agent1_q_value.append(value)
    self.buffer.agent1_n_q_value.append(next_value)
    self.buffer.agent1_prob.append(log_prob.mean())
    self.buffer.agent1_returns.append(returns)
    self.buffer.agent1_msg.append(msg1.cpu())
    self.buffer.agent1_next_prob.append(next_log_prob.mean())
    self.buffer.agent1_action.append(action)
  def learn(self,state,next_state,reward,done,next_value,message):
    next_state        = torch.from_numpy(next_state).float().to(self.device)
    state             = torch.from_numpy(state).float().to(self.device)
    reward            = torch.tensor(reward , dtype = torch.float32).to(self.device)
    done              = torch.tensor(done   , dtype = torch.float32).to(self.device)
    action            = self.choose_action(state,message)
    next_action       = self.choose_action(next_state,message)
    message           = self.message(state,action,message)
    value             = self.q_value(state,action,reward)
    returns           = self.ppo_iter(reward.item(),value,next_value,done)
    returns           = torch.tensor(returns ,dtype = torch.float32).to(self.device)
    advantage         = returns - value -next_value
    log_prob          = torch.log(action).to(self.device)
    next_log          = torch.log(next_action).to(self.device)
    ratio             = (next_log - log_prob).exp()
    s1                = ratio * advantage
    s2                = torch.clamp(ratio,0.8,1.2)
    actor_loss        = torch.min(s1,s2).mean()
    critic_loss       = (returns - value)**2
    critic_loss       = torch.mean(critic_loss)
    actor_loss        = actor_loss
    loss              = actor_loss + 0.5*critic_loss
    mgn_loss          = message * log_prob * value
    mgn_loss          = torch.mean(mgn_loss)
    mgn_loss          = torch.tensor(mgn_loss,requires_grad=True).to(self.device)
    torch.save(self.actor.state_dict(),"memory/Agent1/actor.pth")
    torch.save(self.critic.state_dict(),"memory/Agent1/critic.pth")
    torch.save(self.mgn.state_dict(),"memory/Agent1/mgn.pth")
    self.appending(loss,actor_loss,critic_loss,mgn_loss,value,next_value,log_prob,returns,next_log,action,message)
    self.actor_optim.zero_grad()
    self.critic_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.step()
    self.mgn_optim.zero_grad()
    mgn_loss.backward()
    self.mgn_optim.step()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
from torch.autograd import variable as v
import sys
sys.path.append('./')
from Networks.Actor import Actor
from Networks.Critic import Critic
from Networks.mgn import MGN

class Agent2:
  def __init__(self,state_size,action_size,message_size,buffer):
    self.state_size = state_size
    self.action_size  = action_size
    self.message_size = message_size
    self.gamma        = 0.99
    self.lamda        = 0.95
    self.lr1          = 0.0000009
    self.lr2          = 0.0000007
    self.buffer       = buffer
    self.device       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor        = Actor(self.state_size,self.action_size).to(self.device)
    self.critic       = Critic(self.state_size,self.action_size,self.message_size).to(self.device)
    self.mgn          = MGN(self.state_size,self.action_size,self.message_size).to(self.device)
    self.actor_optim  = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim = optim.Adam(self.critic.parameters() ,lr = self.lr2)
    self.mgn_optim    = optim.Adam(self.mgn.parameters(),lr = self.lr2)
  def choose_action(self,observation,message):
    act               = self.actor(observation,message).to(self.device)
    return act
  def q_value(self,observation,action,reward):
    reward            = torch.tensor([reward.item(),0],dtype = torch.float32).to(self.device)
    value             = self.critic(observation,action,reward).to(self.device)
    return value
  def message(self,observation,action,message):
    message           = self.mgn(observation,action,message).to(self.device)
    return message
  def ppo_iter(self,reward,value,next_value,done):
    returns   = []
    gae       = 0
    for i in range(5):
      delta   = reward + self.gamma *(1-done)*   next_value - value
      gae     = delta + self.gamma*self.lamda*gae*(1-done)
      returns.insert(0,gae + value+next_value)
    return returns
  def appending(self,loss,actor_loss,critic_loss,mgn,value,next_value,log_prob,returns,next_log_prob,action,msg1):
    self.buffer.agent2_mgn_loss.append(mgn)
    self.buffer.agent2_actor_loss.append(actor_loss)
    self.buffer.agent2_critic_loss.append(critic_loss)
    self.buffer.agent2_loss.append(loss)
    self.buffer.agent2_q_value.append(value)
    self.buffer.agent2_n_q_value.append(next_value)
    self.buffer.agent2_prob.append(log_prob.mean())
    self.buffer.agent2_returns.append(returns)
    self.buffer.agent2_msg.append(msg1.cpu())
    self.buffer.agent2_next_prob.append(next_log_prob.mean())
    self.buffer.agent2_action.append(action)
  def learn(self,state,next_state,reward,done,next_value,message):
    next_state        = torch.from_numpy(next_state).float().to(self.device)
    state             = torch.from_numpy(state).float().to(self.device)
    reward            = torch.tensor(reward , dtype = torch.float32).to(self.device)
    done              = torch.tensor(done   , dtype = torch.float32).to(self.device)
    action            = self.choose_action(state,message)
    next_action       = self.choose_action(next_state,message)
    message           = self.message(state,action,message)
    value             = self.q_value(state,action,reward)
    returns           = self.ppo_iter(reward.item(),value,next_value,done)
    returns           = torch.tensor(returns ,dtype = torch.float32).to(self.device)
    advantage         = returns - value -next_value
    log_prob          = torch.log(action).to(self.device)
    next_log          = torch.log(next_action).to(self.device)
    ratio             = (next_log - log_prob).exp()
    s1                = ratio * advantage
    s2                = torch.clamp(ratio,0.8,1.2)
    actor_loss        = torch.min(s1,s2).mean()
    critic_loss       = (returns - value)**2
    critic_loss       = torch.mean(critic_loss)
    actor_loss        = actor_loss
    loss              = actor_loss + 0.5*critic_loss
    mgn_loss          = message * log_prob * value
    mgn_loss          = torch.mean(mgn_loss)
    #mgn_loss          = v(mgn_loss,requires_grad=True).to(self.device)
    mgn_loss          = torch.tensor(mgn_loss,requires_grad=True).to(self.device)
    torch.save(self.actor.state_dict(),"memory/Agent2/actor.pth")
    torch.save(self.critic.state_dict(),"memory/Agent2/critic.pth")
    torch.save(self.mgn.state_dict(),"memory/Agent2/mgn.pth")
    self.appending(loss,actor_loss,critic_loss,mgn_loss,value,next_value,log_prob,returns,next_log,action,message)
    self.actor_optim.zero_grad()
    self.critic_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.step()
    self.mgn_optim.zero_grad()
    mgn_loss.backward()
    self.mgn_optim.step()

In [ ]:

import matplotlib.pyplot as plt
class Ploting:
  def __init__(self,buffer):
    self.buffer = buffer
    self.agent1_loss()
    self.agent1_networks()
    self.agent1_value()
    self.agent1_policy()
    self.agent2_loss()
    self.agent2_networks()
    self.agent2_value()
    self.agent2_policy()
    self.agent1_reward()
    self.agent2_reward()
  def agent1_loss(self):
    plt.plot(self.buffer.episodes,self.buffer.agent1_mean_loss)
    plt.xlabel("episodes")
    plt.ylabel("loss")
    plt.title("loss vs episodes")
    plt.savefig("memory/ploting/agent1_loss.png")
    plt.close()
  def agent1_reward(self):
    plt.plot(self.buffer.episodes,self.buffer.agent1_mean_reward)
    plt.xlabel("episodes")
    plt.ylabel("loss")
    plt.title("loss vs reward")
    plt.savefig("memory/ploting/agent1_reward.png")
    plt.close()
  def agent2_reward(self):
    plt.plot(self.buffer.episodes,self.buffer.agent2_mean_reward)
    plt.xlabel("episodes")
    plt.ylabel("loss")
    plt.title("loss vs reward")
    plt.savefig("memory/ploting/agent2_reward.png")
    plt.close()
  def agent1_networks(self):
    plt.plot(self.buffer.episodes,self.buffer.agent1_mean_actor,label = "actor_loss")
    plt.plot(self.buffer.episodes,self.buffer.agent1_mean_critic,label = "Critic_loss")
    plt.plot(self.buffer.episodes,self.buffer.agent1_mean_loss,label = "loss")
    plt.xlabel("episode")
    plt.ylabel("loss")
    plt.title("episode_vs_agent1_loss")
    plt.legend()
    plt.savefig("memory/ploting/agent1_network_loss.png")
    plt.close()
  def agent1_value(self):
    plt.plot(self.buffer.episodes,self.buffer.agent1_mean_value,label = "q_value")
    plt.plot(self.buffer.episodes,self.buffer.agent1_meannvalue,label = "next_q_value")
    plt.xlabel("episode")
    plt.ylabel("value")
    plt.title("episode_vs_value")
    plt.legend()
    plt.savefig("memory/ploting/agent1_values.png")
    plt.close()
  def agent1_policy(self):
    plt.plot(self.buffer.episodes,self.buffer.agent1_meanlog,label = "policy")
    plt.plot(self.buffer.episodes,self.buffer.agent1_nextlog,label = "next_policy")
    plt.xlabel("episode")
    plt.ylabel("policy")
    plt.title("episode_vs_policy")
    plt.legend()
    plt.savefig("memory/ploting/agent1_policy.png")
    plt.close()
  def agent2_loss(self):
    plt.plot(self.buffer.episodes,self.buffer.agent2_mean_loss)
    plt.xlabel("episodes")
    plt.ylabel("loss")
    plt.title("loss vs episodes")
    plt.savefig("memory/ploting/agent2_loss.png")
    plt.close()
  def agent2_networks(self):
    plt.plot(self.buffer.episodes,self.buffer.agent2_mean_actor,label = "actor_loss")
    plt.plot(self.buffer.episodes,self.buffer.agent2_mean_critic,label = "Critic_loss")
    plt.plot(self.buffer.episodes,self.buffer.agent2_mean_loss,label = "loss")
    plt.xlabel("episode")
    plt.ylabel("loss")
    plt.title("episode_vs_agent2_loss")
    plt.legend()
    plt.savefig("memory/ploting/agent2_network_loss.png")
    plt.close()
  def agent2_value(self):
    plt.plot(self.buffer.episodes,self.buffer.agent2_mean_value,label = "q_value")
    plt.plot(self.buffer.episodes,self.buffer.agent2_meannvalue,label = "next_q_value")
    plt.xlabel("episode")
    plt.ylabel("value")
    plt.title("episode_vs_value")
    plt.legend()
    plt.savefig("memory/ploting/agent2_values.png")
    plt.close()
  def agent2_policy(self):
    plt.plot(self.buffer.episodes,self.buffer.agent2_meanlog,label = "policy")
    plt.plot(self.buffer.episodes,self.buffer.agent2_nextlog,label = "next_policy")
    plt.xlabel("episode")
    plt.ylabel("policy")
    plt.title("episode_vs_policy")
    plt.legend()
    plt.savefig("memory/ploting/agent2_policy.png")
    plt.close()
  def agent_returns(self):
    plt.plot(self.buffer.episodes,self.buffer.agent1_meanreturn,label = "agent1_returns")
    plt.plot(self.buffer.episodes,self.buffer.agent2_meanreturn,label = "agent2_returns")
    plt.xlabel("episode")
    plt.ylabel("returns")
    plt.title("episode_vs_returns")
    plt.legend()
    plt.savefig("memory/ploting/agent1_returns.png") 
    plt.close()
  def agent_mgn(self):
    plt.plot(self.buffer.episodes,self.buffer.agent1_mean_mgn,label = "agent1_mgn_loss")
    plt.plot(self.buffer.episodes,self.buffer.agent2_mean_mgn,label = "agent2_mgn_loss")
    plt.xlabel("episode")
    plt.ylabel("loss")
    plt.title("episode_vs_agent_mgn_loss")
    plt.legend()
    plt.savefig("memory/ploting/agent1_mgn_loss.png")
    plt.close()
  def loss(self):
    plt.plot(self.buffer.episodes,self.buffer.mean_loss)
    plt.xlabel("episodes")
    plt.ylabel("loss")
    plt.title("loss vs episodes")
    plt.savefig("memory/ploting/loss_vs_episodes.png")
    plt.close()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
from torch.autograd import variable as v
import sys
sys.path.append('./')
from buffer.ploting import Ploting
from Agents.Agent1 import Agent1
from Agents.Agent2 import Agent2
from Networks.Critic import Critic
from make_env import make_env
env = make_env("simple_reference")

class Main:
  def __init__(self,state_size,action_size,message_size,buffer,n_agents,n_games,steps):
    self.state_size = state_size
    self.action_size = action_size
    self.message_size = message_size
    self.n_agents    = n_agents
    self.n_games     = n_games
    self.steps       = steps
    self.gamma  = 0.99
    self.lr     = 0.0000007
    self.buffer = buffer
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.agent1 = Agent1(self.state_size,self.action_size,self.message_size,self.buffer)
    self.agent2 = Agent2(self.state_size,self.action_size,self.message_size,self.buffer)
    self.central= Critic(self.state_size,self.action_size,self.message_size).to(self.device)
    self.optim  = optim.Adam(self.central.parameters() , lr = self.lr)
    self.msg1   = []
    self.msg2   = []
  def choose_action(self,state,msg1,msg2):
    act  = []
    obs1 = torch.from_numpy(state[0]).float().to(self.device)
    obs2 = torch.from_numpy(state[1]).float().to(self.device)
    msg1 = torch.tensor(msg1,dtype = torch.float32).to(self.device)
    msg2 = torch.tensor(msg2,dtype = torch.float32).to(self.device)
    act1 = self.agent1.choose_action(obs1,msg2).cpu()
    act2 = self.agent2.choose_action(obs2,msg1).cpu()
    act.append(act1.detach().numpy())
    act.append(act2.detach().numpy())
    return act
  def next_value(self,next_state,action,r1,r2):
    obs1 = torch.from_numpy(next_state[0]).float().to(self.device)
    obs2 = torch.from_numpy(next_state[1]).float().to(self.device)
    r1   = torch.tensor([r1,0],dtype = torch.float32).to(self.device)
    r2   = torch.tensor([r2,0],dtype = torch.float32).to(self.device)
    nv1  = self.central(obs1,action[0],r1)
    nv2  = self.central(obs2,action[1],r2)
    return nv1,nv2
  def message(self,state,action,msg1,msg2):
    obs1 = torch.from_numpy(state[0]).float().to(self.device)
    obs2 = torch.from_numpy(state[1]).float().to(self.device)
    act1 = torch.tensor(action[0],dtype = torch.float32).to(self.device)
    act2 = torch.tensor(action[1],dtype = torch.float32).to(self.device)
    msg2 = torch.tensor(msg2,dtype = torch.float32).to(self.device)
    msg1 = self.agent1.message(obs1,act1,msg2)
    msg2 = self.agent2.message(obs2,act2,msg1)
    return msg1,msg2
  def update(self,state,next_state,reward,done,msg1,msg2):
    action     = self.choose_action(state,msg1,msg2)
    action     = torch.tensor(action,dtype = torch.float32).to(self.device)
    msg1,msg2  = self.message(state,action,msg1,msg2)
    agnv1,agnv2= self.next_value(next_state,action,reward[0],reward[1])
    agv1       = self.agent1.q_value(torch.from_numpy(state[0]).float().to(self.device),action[0],torch.tensor(reward[0],dtype = torch.float32))
    agv2       = self.agent2.q_value(torch.from_numpy(state[1]).float().to(self.device),action[1],torch.tensor(reward[1],dtype = torch.float32))
    agr1       = torch.tensor(self.agent1.ppo_iter(torch.tensor(reward[0],dtype = torch.float32).to(self.device),agv1,agnv1,done[0]))
    agr2       = torch.tensor(self.agent2.ppo_iter(torch.tensor(reward[1],dtype = torch.float32).to(self.device),agv2,agnv2,done[1]))
    returns    = torch.tensor(agr1 + agr2).to(self.device)
    loss       = returns - agv1-agv2-agnv1-agnv2
    loss       = torch.tensor(loss[0],requires_grad = True).to(self.device)
    self.buffer.loss.append(loss)
    torch.save(self.central.state_dict() , "memory/centeral.pth")
    self.agent1.learn(state[0],next_state[0],reward[0],done[0],agnv1,msg2)
    self.agent2.learn(state[1],next_state[1],reward[1],done[1],agnv2,msg1)
    self.optim.zero_grad()
    loss.backward()
    self.optim.step()
  def mean(self,episode):
    if episode==0:
      self.buffer.mean_loss.append(sum(self.buffer.loss))
      self.buffer.agent1_mean_loss.append(sum(self.buffer.agent1_loss))
      self.buffer.agent1_mean_actor.append(sum(self.buffer.agent1_actor_loss))
      self.buffer.agent1_mean_critic.append(sum(self.buffer.agent1_critic_loss))
      self.buffer.agent1_mean_mgn.append(sum(self.buffer.agent1_mean_loss))    
      self.buffer.agent1_meanlog.append(sum(self.buffer.agent1_prob))
      self.buffer.agent1_nextlog.append(sum(self.buffer.agent1_next_prob))
      self.buffer.agent1_mean_value.append(sum(self.buffer.agent1_q_value))
      self.buffer.agent1_meanreturn.append(sum(self.buffer.agent1_returns))
      self.buffer.agent1_meannvalue.append(sum(self.buffer.agent1_n_q_value))
      self.buffer.agent1_mean_reward.append(sum(self.buffer.agent1_reward))
      self.buffer.agent2_mean_actor.append(sum(self.buffer.agent2_actor_loss))
      self.buffer.agent2_mean_critic.append(sum(self.buffer.agent2_critic_loss))
      self.buffer.agent2_mean_mgn.append(sum(self.buffer.agent2_mean_loss))    
      self.buffer.agent2_meanlog.append(sum(self.buffer.agent2_prob))
      self.buffer.agent2_nextlog.append(sum(self.buffer.agent2_next_prob))
      self.buffer.agent2_mean_value.append(sum(self.buffer.agent2_q_value))
      self.buffer.agent2_meanreturn.append(sum(self.buffer.agent2_returns))
      self.buffer.agent2_meannvalue.append(sum(self.buffer.agent2_n_q_value))
      self.buffer.agent2_mean_reward.append(sum(self.buffer.agent2_reward))
      self.buffer.agent2_mean_loss.append(sum(self.buffer.agent2_loss))
    else:
      self.buffer.mean_loss.append(sum(self.buffer.loss)/len(self.buffer.loss))
      self.buffer.agent1_mean_loss.append(torch.tensor(sum(self.buffer.agent1_loss)/len(self.buffer.agent1_loss)).cpu())
      self.buffer.agent2_mean_loss.append(torch.tensor(sum(self.buffer.agent2_loss)/len(self.buffer.agent2_loss)).cpu())
      self.buffer.agent1_mean_actor.append(torch.tensor(sum(self.buffer.agent1_actor_loss)/len(self.buffer.agent1_actor_loss)).cpu())
      self.buffer.agent1_mean_critic.append(torch.tensor(sum(self.buffer.agent1_critic_loss)/len(self.buffer.agent1_critic_loss)).cpu())
      self.buffer.agent1_mean_mgn.append(sum(self.buffer.agent1_mgn_loss)/len(self.buffer.agent1_mgn_loss))    
      self.buffer.agent1_meanlog.append(torch.tensor(sum(self.buffer.agent1_prob)/len(self.buffer.agent1_prob)).cpu())
      self.buffer.agent1_nextlog.append(torch.tensor(sum(self.buffer.agent1_next_prob)/len(self.buffer.agent1_next_prob)).cpu())
      self.buffer.agent1_mean_value.append(torch.tensor(sum(self.buffer.agent1_q_value)/len(self.buffer.agent1_q_value)).cpu())
      self.buffer.agent1_meanreturn.append(sum(self.buffer.agent1_returns)/len(self.buffer.agent1_returns))
      self.buffer.agent1_meannvalue.append(torch.tensor(sum(self.buffer.agent1_n_q_value)/len(self.buffer.agent1_n_q_value)).cpu())
      self.buffer.agent1_mean_reward.append(torch.tensor(sum(self.buffer.agent1_reward)/len(self.buffer.agent1_reward)).cpu())
      self.buffer.agent2_mean_actor.append(torch.tensor(sum(self.buffer.agent2_actor_loss)/len(self.buffer.agent2_actor_loss)).cpu())
      self.buffer.agent2_mean_critic.append(torch.tensor(sum(self.buffer.agent2_critic_loss)/len(self.buffer.agent2_critic_loss)).cpu())
      self.buffer.agent2_mean_mgn.append(sum(self.buffer.agent2_mgn_loss)/len(self.buffer.agent2_mgn_loss))    
      self.buffer.agent2_meanlog.append(torch.tensor(sum(self.buffer.agent2_prob)/len(self.buffer.agent2_prob)).cpu())
      self.buffer.agent2_nextlog.append(torch.tensor(sum(self.buffer.agent2_next_prob)/len(self.buffer.agent2_next_prob)).cpu())
      self.buffer.agent2_mean_value.append(torch.tensor(sum(self.buffer.agent2_q_value)/len(self.buffer.agent2_q_value)).cpu())
      self.buffer.agent2_meanreturn.append(sum(self.buffer.agent2_returns)/len(self.buffer.agent2_returns))
      self.buffer.agent2_meannvalue.append(torch.tensor(sum(self.buffer.agent2_n_q_value)/len(self.buffer.agent2_n_q_value)).cpu())
      self.buffer.agent2_mean_reward.append(torch.tensor(sum(self.buffer.agent2_reward)/len(self.buffer.agent2_reward)).cpu())
  def clear(self):
    self.buffer.loss = []
    self.buffer.agent2_actor_loss = []
    self.buffer.agent2_critic_loss = []
    self.buffer.agent2_prob = []
    self.buffer.agent2_next_prob= []
    self.buffer.agent2_q_value =[]
    self.buffer.agent2_returns  = []
    self.buffer.agent2_n_q_value = []
    self.buffer.agent2_reward = []
    self.buffer.agent1_actor_loss = []
    self.buffer.agent1_critic_loss = []
    self.buffer.agent1_prob = []
    self.buffer.agent1_next_prob= []
    self.buffer.agent1_q_value =[]
    self.buffer.agent1_returns  = []
    self.buffer.agent1_n_q_value = []
    self.buffer.agent1_reward = []
  def run(self):
    for i in range(self.n_games):
      state = env.reset()
      score = [0,0]
      done  = [False ,False]
      self.buffer.episodes.append(i)
      self.mean(i)
      self.clear()
      plts = Ploting(self.buffer)
      print("episode:",i,",","agent1_reward:",self.buffer.agent1_mean_reward[i],",","agent2_reward:",self.buffer.agent1_mean_reward[i])
      for step in range(self.steps):
        if i==0 & step==0:
          msg1 = torch.zeros(1)
          msg2 = torch.zeros(1)
          action  = self.choose_action(state,msg1,msg2)
          msg1,msg2 = self.message(state,action,msg1,msg2)
          self.msg1.append(msg1)
          self.msg2.append(msg2)
        else:
          msg1 = self.msg1[0]
          msg2 = self.msg2[0]
          action= self.choose_action(state,msg1,msg2)
          self.msg1 = []
          self.msg2 = []
          self.msg1.append(msg1)
          self.msg2.append(msg2)
        next_state,reward,done,info = env.step(action)
        self.buffer.agent1_reward.append(reward[0])
        self.buffer.agent2_reward.append(reward[1]) 
        self.update(state,next_state,reward,done,msg1,msg2)
        if done:
          state = next_state
          score += reward
        else:
          self.update(state,next_state,reward,done,msg1,msg2)
          state = next_state
          score += reward
          print("completed")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
from torch.autograd import variable as v
import sys
sys.path.append('./')
from main import Main
from buffer.buffer import Buffer
if __name__ =="__main__":
  buffer = Buffer()
  main = Main(21,5,5,buffer,2,100,100)
  main.run()

episode: 0 , agent1_reward: 0 , agent2_reward: 0
-1.553622198589335


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-p

-1.5552437553048892
-1.5574720664116666
-1.560243133334449
-1.563486442280751
-1.5671650870975342
-1.5712548350425593
-1.575708614987663
-1.5805063831971078
-1.585637538772731
-1.5910754824536282
-1.5968225094950492
-1.6028706094843563
-1.6092023937597704
-1.615810686905632
-1.6226832369141393
-1.629808164875258
-1.6371783612907491
-1.6447677525701798
-1.6525690407020137
-1.660574120979126
-1.668774261819734
-1.6771602222564292
-1.6857404116049477
-1.6945011060138149
-1.7034422902449267
-1.7125555445389677
-1.7218239864205553
-1.7312551031531074
-1.7408526679834986
-1.750617167457727
-1.7605535838405626
-1.7706662732265088
-1.7809529498255987
-1.7914103271914932
-1.8020406591143983
-1.8128699037974938
-1.8238899721539599
-1.8351069742334845
-1.8465308806247736
-1.8581455384084538
-1.8699656762224794
-1.8820087371382062
-1.8942739829058544
-1.9067918028331705
-1.919574235517134
-1.9326094276551957
-1.9459036871133235
-1.9594679342609187
-1.9732840374206293
-1.9873582803965744
-2.0016587

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru

episode: 1 , agent1_reward: tensor(-3.2428, dtype=torch.float64) , agent2_reward: tensor(-3.2428, dtype=torch.float64)
-2.240327307567644
-2.239347663459869
-2.2381252890670824
-2.236792165254176
-2.2353826777659305
-2.233968045403971
-2.2326175461335596
-2.231376724489004
-2.2302927774605124
-2.229352862154497
-2.2285740104328893
-2.227960545743552
-2.2275485156331825
-2.227311666594101
-2.2273009227916734
-2.2274853933801024
-2.227878963567726
-2.228485923290767
-2.22932957990577
-2.230389520509738
-2.2316711694896694
-2.23317850045302
-2.23489886547968
-2.2368221368768806
-2.2388781474241584
-2.2411180846235665
-2.243588587520553
-2.246269313732267
-2.2491601117138678
-2.2521851158672725
-2.2554565033465703
-2.2589545013620134
-2.2626600248762845
-2.2665961018324863
-2.270712781061454
-2.2749414477817647
-2.279274956344052
-2.283788948011229
-2.2885539706782803
-2.2935262978805087
-2.2987679863220416
-2.304158558173291
-2.309759188226796
-2.3155749840329
-2.3216327768631193
-2.32787

In [ ]:
buffer.agent2_mean_reward

In [ ]:
msg1 = buffer.agent1_msg
msg2 = buffer.agent2_msg

In [ ]:
dict = {"msg1":msg1,"msg2":msg2}

In [ ]:
import pandas as pd  
df = pd.DataFrame(dict) 
df.to_csv('communication.csv')

In [ ]:
def csvs(buffer):
  msg1 = buffer.agent1_msg
  msg2 = buffer.agent2_msg
  d1   = {"msg1":msg1,"msg2":msg2}
  d1 = pd.DataFrame(dict) 
  d1.to_csv('communication.csv')
 

In [ ]:
d = torch.tensor(3.14)

In [ ]:
l = nn.Linear(0,32)

In [ ]:
l(d)